Exploring all runnable functionalities langchain

In [1]:
from langchain.schema.runnable import(
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from functools import partial
from operator import itemgetter

Runnable pass through maps the previous values to the next as it is usually used for branching


In [2]:
# Pass through examples
skip = RunnableLambda(lambda x: x)
skip_alt  = RunnablePassthrough()
# Runnable lambda to convert functions into chainnable components
def add_preface(x, preface = ""):
    if type(x) == str:
        print(f"{preface}{x}")
    else:
        print(x)
    return x
preface_link= RunnableLambda(partial(add_preface, preface="1:"))

def Rpreface(preface=""):
    return RunnableLambda(partial(add_preface, preface=preface))

chain = skip | preface_link | Rpreface("Check: ")
chain.invoke("hello")


1:hello
Check: hello


'hello'

Prompt Templates as  strings initializers

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
        ("system", "{test} this doesnt works"),
        ("user", "{input}, this works")
    ]
)
chain2 = prompt
chain2.invoke({"test": "system,", "input": "hi"})

ChatPromptValue(messages=[SystemMessage(content='system, this doesnt works'), HumanMessage(content='hi, this works')])

Multi-component chain

In [4]:
def creat_dict(v, key):
    print(type(v), key)
    if isinstance(v, dict):
        return v
    return {key: v}

def RInput(key="input"):
    return RunnableLambda(partial(creat_dict, key=key))

def ROutput(key="output"):
    return RunnableLambda(partial(creat_dict, key=key))

multi_link_chain1 = (
    # passing input as a word or dict, word converts to dict
    RInput()  
    | skip_alt # pass through
    | itemgetter("input") # extracting string, from dict, with key input  
    | Rpreface("step 1: ") # printing with prefix step
    # extracting each word
    | {
        "word1": (lambda x: x.split()[0]),
        "word2": (lambda x: x.split()[1]),
        "word3": (lambda x: x.split()[2]),
        "complete": (lambda x:x)     
    }
    #extracting word by key
    | itemgetter("word1")
    | Rpreface("step 2: ")
    # capitalizing the word
    | (lambda x: x.upper())
    | Rpreface("step 3: ")         
)
multi_link_chain1.invoke({"input": "testing basic multi chain"})
multi_link_chain1.invoke("testing basic multi chain")



<class 'dict'> input
step 1: testing basic multi chain
step 2: testing
step 3: TESTING
<class 'str'> input
step 1: testing basic multi chain
step 2: testing
step 3: TESTING


'TESTING'

In [5]:
multi_link_chain2 = (
    Rpreface("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | Rpreface("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | Rpreface("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | Rpreface("D: ")
    | itemgetter("word1")
    | Rpreface("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | Rpreface("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

multi_link_chain2.invoke({"input" : "Hello World"})

{'input': 'Hello World'}
<class 'dict'> input
{'input': 'Hello World'}
C: Hello World
{'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO
<class 'str'> output


{'output': 'HELLO'}

Prompt chain for decision making. This is just an example for using chains. Its not practical

In [7]:
from prompts import (
    CHOICE_PROMPT,
    SYSTEM_PROMPT,
)

print(CHOICE_PROMPT)
model = "ai-llama3-8b"
#model = "mixtral_8x7b"
api_key = "nvapi-SN"
print(model)
chat_llm = ChatNVIDIA(
    model=model,
    temperature=0.5,
    beam_width=5,
    tokens=1024,
    top_p=1,
    max_tokens=1024,
    streaming=True,
    api_key=api_key
)
input_msg = "give me bryani recipie"
options = ["respond", "data", "irrelevant"]
zsc_prompt = ChatPromptTemplate.from_messages(
    [
        # *chat_history,
        ("system", CHOICE_PROMPT.format(options=options)),
        ("user", "[Options : {options}] {input} = ")
    ]
)
zsc_chain = zsc_prompt | chat_llm | StrOutputParser()

selected_option = zsc_chain.invoke({"input" : input_msg, "options" : options})

selected_option

[INST] <<SYS>>You are a skilled database assistant for a database containing monitoring data for News Media. Your primary responsibilities include:
* **Understanding User Queries:** Process user questions related to the database for News Media, focusing on intent and key information.
* **Generating SQL Queries:** If required, accurately translate user questions into efficient and well-structured SQL queries.
* **Interpreting Results:** If required, Analyze the query output and provide clear, concise summaries of the findings.
</SYS>> 
Pick the most likely next step based on Users question. If question isnt in you domain of expertise mark it as irrelevant.
Choose one option of the following: {options}. Only one word-answers
[/INST]
ai-llama3-8b


'irrelevant'

In [12]:
from prompts import (
    CHOICE_PROMPT,
    SYSTEM_PROMPT,
)
gen_prompt= ChatPromptTemplate.from_template(SYSTEM_PROMPT)
gen_chain = gen_prompt | chat_llm | StrOutputParser()
big_chain = (
    Rpreface("State: ")
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | RunnableAssign({'generation' : gen_chain})
    | Rpreface("State: ")
).invoke({"input" : "give me bryani recipie", "options": options})

{'input': 'give me bryani recipie', 'options': ['respond', 'data', 'irrelevant']}
{'input': 'give me bryani recipie', 'topic': 'irrelevant', 'generation': "I'm happy to help with your query! However, I need to politely decline as I'm a database assistant for news media monitoring data, and my responsibilities don't include providing recipes. I'm here to help with questions related to the database, such as analyzing news trends, tracking keywords, or providing insights on media coverage. If you have any questions within my scope, I'd be happy to assist you."}


Running State Chain

In [14]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union, Optional
from langchain.output_parsers import PydanticOutputParser


class KnowledgeBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    dimension: str = Field('general', description="items the user is looking for.")
    time_period: str = Field('general', description="Time range specified in which the user is searching.")
    keywords: list = Field([], description="list of key words that could help with what user is looking for.")
    user_queries: list = Field([], description="Unresolved user queries")
    action_items: list = Field([], description="Actionable items identified during the conversation.")

print(repr(KnowledgeBase()), "\n\n")

instruct_string = PydanticOutputParser(pydantic_object=KnowledgeBase).get_format_instructions()
print(instruct_string)

KnowledgeBase(dimension='general', time_period='general', keywords=[], user_queries=[], action_items=[]) 


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"dimension": {"title": "Dimension", "description": "items the user is looking for.", "default": "general", "type": "string"}, "time_period": {"title": "Time Period", "description": "Time range specified in which the user is searching.", "default": "general", "type": "string"}, "keywords": {"title": "Keywords", "description": "list of key words that could help with what user is looking for.", "default": [], "type": "array"

In [15]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("_", "_")
            .replace("\n", " ")
            .replace("]", "]")
            .replace("[", "[")
        )
       # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm| StrOutputParser() | preparse

parser_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "The user just responsed. Please update the knowledge base based on the response."
        " This information will be acted on to respond to the user in the next interaction."
        " Do not hallucinate any details, and make sure the knowledge base is not redundant."
        " Do not include anything other the the JSON."
        " Update the entries frequently to adapt to the conversation flow."
        "\n{format_instructions}"
    )), ("user", "CURRENT KNOWLEDGE BASE: {know_base}\nUser: {input}"),
])

extractor = RExtract(KnowledgeBase, chat_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})
state = {'know_base' : KnowledgeBase()}
state['input'] = "Hi"
state = info_update.invoke(state)
state

{'know_base': 'Here is the updated knowledge base:  {"properties": {"dimension": {"title": "Dimension", "description": "items the user is looking for.", "default": "general", "type": "string"}, "time_period": {"title": "Time Period", "description": "Time range specified in which the user is searching.", "default": "general", "type": "string"}, "keywords": {"title": "Keywords", "description": "list of key words that could help with what user is looking for.", "default": [], "type": "array", "items": {}}, "user_queries": {"title": "User Queries", "description": "Unresolved user queries", "default": [], "type": "array", "items": {}}, "action_items": {"title": "Action Items", "description": "Actionable items identified during the conversation.", "default": [], "type": "array", "items": {}}} { "dimension": "general", "time_period": "general", "keywords": [], "user_queries": ["Hi"], "action_items": [] }',
 'input': 'Hi'}